# LangChain Expression Language (LCEL)

In [1]:
import os
from aisModels_lc import GroqChatLLM

os.system("clear")

llm = GroqChatLLM()
print(llm.invoke("hello").content)

Hello. How can I assist you today?
Hello! How are you today? Is there something I can help you with or would you like to chat?
Hello. How can I assist you today?


In [2]:
%%script true
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
#!pip install pydantic==1.10.8

In [4]:
from langchain.prompts import ChatPromptTemplate
#from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

## Simple Chain

In [5]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = llm
output_parser = StrOutputParser()

In [6]:
chain = prompt | model | output_parser

In [7]:
chain.invoke({"topic": "bears"})

'Why did the bear go to the doctor? \n\nBecause it had a grizzly cough.'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [8]:
#from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [9]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    #embedding=OpenAIEmbeddings()
    embedding=OllamaEmbeddings(model="nomic-embed-text")
    )
retriever = vectorstore.as_retriever()

/home/tjamil/anaconda3/envs/llm/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [10]:
retriever.get_relevant_documents("where did harrison work?")

/tmp/ipykernel_5498/3310280720.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retriever.get_relevant_documents("where did harrison work?")


[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [11]:
retriever.get_relevant_documents("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [12]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [13]:
from langchain.schema.runnable import RunnableMap

In [14]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [15]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [16]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [17]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind

and OpenAI Functions

In [18]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [19]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = GroqChatLLM(temperature=0).bind(functions=functions)

In [20]:
runnable = prompt | model

In [21]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code": "SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 233, 'total_tokens': 250, 'completion_time': 0.068, 'prompt_time': 0.054340276, 'queue_time': 0.0053347889999999995, 'total_time': 0.122340276}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'function_call', 'logprobs': None}, id='run-b6d61c18-a396-440f-bd68-1d7f00458c10-0', usage_metadata={'input_tokens': 233, 'output_tokens': 17, 'total_tokens': 250})

In [22]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [23]:
model = model.bind(functions=functions)

In [24]:
runnable = prompt | model

In [25]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name": "patriots"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 299, 'total_tokens': 317, 'completion_time': 0.072, 'prompt_time': 0.073978484, 'queue_time': 0.00503075900000001, 'total_time': 0.145978484}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'function_call', 'logprobs': None}, id='run-232048ea-0b23-40e8-ad36-4f9e99df67f3-0', usage_metadata={'input_tokens': 299, 'output_tokens': 18, 'total_tokens': 317})

## Interface

In [26]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = llm#ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [27]:
chain.invoke({"topic": "bears"})

'Why did the bear go to the doctor? \n\nBecause it had a grizzly cough.'

In [28]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

['Why did the bear go to the doctor? \n\nBecause it had a grizzly cough.',
 'Why did the frog go to the doctor? \n\nBecause it had a "ribbiting" cough.']

In [29]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 did
 the
 bear
 go
 to
 the
 doctor
?
 


Because
 it
 had
 a
 gr
izzly
 cough
.



In [30]:
response = await chain.ainvoke({"topic": "bears"})
response

'Why did the bear go to the doctor? \n\nBecause it had a grizzly cough.'